#DuckDB Playground

A notebook to playaround with and get familiar with DuckDB. Duck DB runs inside your python environment.

**Introduction to DuckDB:**

DuckDB is an in-process SQL OLAP (Online Analytical Processing) database designed for fast query performance, ease of use, and seamless integration with Python, R, and other programming environments. It offers several key benefits:

- **Fast Query Execution**: DuckDB uses vectorized query execution (fast joins), enabling high-speed analytics on large datasets.
- **No Server Overhead**: As an in-process database, it runs directly within your application, eliminating the need for a separate database server.
- **Zero Configuration**: It is easy to set up and requires no complex configurations, making it ideal for quick prototyping and analysis.
- **Efficient Storage**: DuckDB supports efficient storage formats like Parquet, allowing you to query large datasets directly from these files without the need for preprocessing.
- **Columnar Storage**: DuckDB uses a columnar storage format similar to Parquet, optimizing both memory usage and query performance, especially for analytical workloads.
- **Parallel Execution**: DuckDB performs parallel query execution across multiple CPU cores by default, making it highly efficient for running complex analytical queries.
- **Lightweight & Portable**: Its lightweight nature allows it to be embedded within applications, making it highly portable for different environments.
- **Python UDFs (User Defined Functions)**: You can write Python functions and use them directly in SQL queries, allowing for custom operations that SQL does not support natively.

This makes DuckDB an excellent choice for data exploration, ad-hoc analysis, and integrating into data science workflows.

https://duckdb.org/


##Load Data

In [ ]:
import pandas as pd
from google.colab import drive
from tabulate import tabulate
import duckdb

drive.mount('/content/drive')

#Load data into data frame
df = pd.read_csv('/content/drive/My Drive/Learning/Datasets/Airbnb_Open_Data.csv')

# Make all headers lowercase and use underscores in spaces
df.columns = df.columns.str.lower().str.replace(' ', '_')

#View all columns
pd.set_option('display.max_columns', None)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-a0ad9fd89cd8>:9: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/My Drive/Learning/Datasets/Airbnb_Open_Data.csv')


,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,country_code,instant_bookable,cancellation_policy,room_type,construction_year,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,US,False,strict,Private room,2020.0,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,US,False,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,US,True,flexible,Private room,2005.0,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,US,True,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,US,False,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


###Clean data

Correct datatypes and remove $

In [ ]:
# Remove $ from price and service_fee and convert to integers
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float).fillna(0).astype(int)
df['service_fee'] = df['service_fee'].str.replace('$', '').str.replace(',', '').astype(float).fillna(0).astype(int)

# Convert construction_year, minimum_nights, number_of_reviews, availability_365 to integer
df['construction_year'] = df['construction_year'].fillna(0).astype(int)
df['minimum_nights'] = df['minimum_nights'].fillna(0).astype(int)
df['number_of_reviews'] = df['number_of_reviews'].fillna(0).astype(int)
df['availability_365'] = df['availability_365'].fillna(0).astype(int)
df['calculated_host_listings_count'] = df['calculated_host_listings_count'].fillna(0).astype(int)

# Convert review_rate_number to object
df['review_rate_number'] = df['review_rate_number'].fillna(0).astype(object)


In [ ]:
df.head()

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,country_code,instant_bookable,cancellation_policy,room_type,construction_year,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,US,False,strict,Private room,2020,966,193,10,9,10/19/2021,0.21,4.0,6,286,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,US,False,moderate,Entire home/apt,2007,142,28,30,45,5/21/2022,0.38,4.0,2,228,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,US,True,flexible,Private room,2005,620,124,3,0,NaN,NaN,5.0,1,352,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,US,True,moderate,Entire home/apt,2005,368,74,30,270,7/5/2019,4.64,4.0,1,322,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,US,False,moderate,Entire home/apt,2009,204,41,10,9,11/19/2018,0.10,3.0,1,289,"Please no smoking in the house, porch or on th...",NaN


##Install DuckDB & ingest data

In [ ]:
#!pip install duckdb

In [ ]:
# Create a DuckDB database
con = duckdb.connect('airbnb.db')

# Create a table in the database from the pandas DataFrame - Super cool!
con.execute("CREATE OR REPLACE TABLE airbnb_data AS SELECT * FROM df")

In [ ]:
# View the first five rows of the table
result = con.execute("SELECT * FROM airbnb_data LIMIT 5").df()
print(tabulate(result, headers='keys', tablefmt='psql'))


+----+---------+--------------------------------------------------+-------------+--------------------------+-------------+-----------------------+-----------------+---------+----------+---------------+----------------+--------------------+-----------------------+-----------------+---------------------+---------+---------------+------------------+---------------------+---------------+---------------------+----------------------+----------------------------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|    |      id | name                                             |     host_id | host_identit

Let's describe the data in the table.

In [ ]:
result = con.execute("DESCRIBE airbnb_data").df()
print(tabulate(result, headers='keys', tablefmt='psql'))

+----+--------------------------------+---------------+--------+-------+-----------+---------+
|    | column_name                    | column_type   | null   | key   | default   | extra   |
|----+--------------------------------+---------------+--------+-------+-----------+---------|
|  0 | id                             | BIGINT        | YES    |       |           |         |
|  1 | name                           | VARCHAR       | YES    |       |           |         |
|  2 | host_id                        | BIGINT        | YES    |       |           |         |
|  3 | host_identity_verified         | VARCHAR       | YES    |       |           |         |
|  4 | host_name                      | VARCHAR       | YES    |       |           |         |
|  5 | neighbourhood_group            | VARCHAR       | YES    |       |           |         |
|  6 | neighbourhood                  | VARCHAR       | YES    |       |           |         |
|  7 | lat                            | DOUBLE    

##SQL Querries

How many rows are in the table?

In [ ]:
# This line executes an SQL query on the DuckDB connection con
result = con.execute("SELECT count(*) FROM airbnb_data").df()
print(tabulate(result, headers='keys', tablefmt='psql'))


+----+----------------+
|    |   count_star() |
|----+----------------|
|  0 |         102599 |
+----+----------------+


How many listings in new york are priced over $500?

In [ ]:
result = con.execute("SELECT count(*) FROM airbnb_data WHERE price > 500 AND neighbourhood_group = 'Manhattan'").df()
print(tabulate(result, headers='keys', tablefmt='psql'))

+----+----------------+
|    |   count_star() |
|----+----------------|
|  0 |          26442 |
+----+----------------+


Count the number of nulls in the `name` field.

In [ ]:

result = con.execute("SELECT COUNT(*) FROM airbnb_data WHERE name IS NULL").df()
print(tabulate(result, headers='keys', tablefmt='psql'))

+----+----------------+
|    |   count_star() |
|----+----------------|
|  0 |            250 |
+----+----------------+


Show the tables in the database.

In [ ]:
result = con.execute("SHOW TABLES").df()
print(tabulate(result, headers='keys', tablefmt='psql'))

+----+-------------+
|    | name        |
|----+-------------|
|  0 | airbnb_data |
+----+-------------+


## UDF in SQL

Create a UDF and run it in the SQL querry. Note that DuckDB requires you to define the data type.

In [ ]:
from duckdb.typing import DOUBLE

def total_price(price: DOUBLE, minimum_nights: DOUBLE) -> DOUBLE:
  """Calculates the total price of an Airbnb listing.

  Args:
    price: The price per night for the listing.
    minimum_nights: The minimum number of nights required to book.

  Returns:
    The total price of the Airbnb listing.
  """
  return price * minimum_nights

In [ ]:
# Register the UDF with DuckDB, explicitly setting the return type
con.create_function("total_price", total_price, return_type=DOUBLE)

In [ ]:
# Use the UDF in a SQL query!!! Limit to 10 rows
result = con.execute("SELECT total_price(price, minimum_nights) AS total_price FROM airbnb_data LIMIT 10").df()
print(tabulate(result, headers='keys', tablefmt='psql'))

+----+---------------+
|    |   total_price |
|----+---------------|
|  0 |          9660 |
|  1 |          4260 |
|  2 |          1860 |
|  3 |         11040 |
|  4 |          2040 |
|  5 |          1731 |
|  6 |          3195 |
|  7 |         47700 |
|  8 |          2036 |
|  9 |           582 |
+----+---------------+
